In [8]:
import json
import gzip
import zipfile
import os
import time
import logging

import numpy as np
import pandas as pd

from operator import itemgetter
from pandas import DataFrame

In [25]:
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
logger = logging.getLogger('phywhgs_logger')
logger.setLevel(logging.INFO)
logging.basicConfig(filename='./phylowgs.common.format/phylowgs.log', level=logging.DEBUG, filemode='w', datefmt="%Y-%m-%d %H:%M:%S")

In [19]:
pp_table = pd.read_table('./team_project_data/Data/pp_table.txt', sep='\t')

In [20]:
evolved_samples = [(s.split('.')[0], s) for s in os.listdir('./phylowgs.results/')]
evolved_samples.sort()

In [5]:
def obtain_subclonal_structure(trees, purity):
    chain_likelihoods = np.array([(i, trees[str(i)]['llh']) for i in trees], 
                                 dtype=[('chain', (np.str_, 10)), ('llh', np.float128)])
    maxllh_chain = chain_likelihoods[chain_likelihoods['llh'].argmax()]
    pop_dict = trees[maxllh_chain['chain']]['populations']
    relevant_data = [[cl, pop_dict[cl]['num_ssms'], pop_dict[cl]['cellular_prevalence'][0], pop_dict[cl]['cellular_prevalence'][0] / purity] for cl in pop_dict if pop_dict[cl]['num_ssms']> 0]
    return maxllh_chain, relevant_data

In [6]:
def obtain_mutation_assignments(mutass, ccfs, ssms):
    mut_ccf = lambda mut : [ccfs[mut[1]]]
    mut = np.array([[mut_id, cluster] for cluster in mutass for mut_id in mutass[cluster]['ssms']], dtype=np.str_)
    mut = np.hstack((mut,map(mut_ccf, mut))) 
    mut = mut[mut[:,0].argsort()] # sort for stacking
    ssms = ssms[ssms[:,0].argsort()] #sort for stacking   
    relevant_data = np.concatenate((ssms[:,[0,3,4]], mut[:,1:]), axis=1)    
    return relevant_data

In [26]:
failed_samples = list()
PATH_SUBCLONAL = './phylowgs.common.format/subclonal_structure'
PATH_MUTASS = './phylowgs.common.format/mutation_assignments'

if not os.path.exists(PATH_SUBCLONAL):        
    os.makedirs(PATH_SUBCLONAL)
    
if not os.path.exists(PATH_MUTASS):
    os.makedirs(PATH_MUTASS)

for i, (sample, results_folder) in enumerate(evolved_samples):
    subclonal_structure = None
    maxllh_chain = None
    mutation_assignments = None
    purity = pp_table[pp_table['sample'] == sample]['purity'].values[0]
    
    try:
        start_time = time.time()
        
        with gzip.open("./phylowgs.results/{}/test_results/{}.summ.json.gz".format(results_folder, sample), "rb") as f:
            d = json.loads(f.read().decode("ascii"))
            maxllh_chain, subclonal_structure = obtain_subclonal_structure(trees=d['trees'], purity=purity)

        with zipfile.ZipFile("./phylowgs.results/{}/test_results/{}.mutass.zip".format(results_folder, sample), "r") as f:
            d = json.loads(f.read("{}.json".format(maxllh_chain['chain'])))
            ccfs = {cl[0]: cl[-1] for cl in subclonal_structure}
            chrom_pos = lambda cp: [int(cp[1].split('_')[0]), int(cp[1].split('_')[1])]

            ssms = np.loadtxt('./phylowgs.results/{}/ssm_data.txt'.format(results_folder), 
                              delimiter='\t', skiprows=1, usecols=(0,1), dtype=(np.str_, np.str_)) 
            ssms = np.concatenate((ssms,map(chrom_pos, ssms)), axis=1)
            ssms = np.hstack((np.arange(ssms.shape[0])[:, np.newaxis], ssms))        
            mutation_assignments = obtain_mutation_assignments(mutass=d['mut_assignments'], ccfs=ccfs, ssms=ssms)


        ss_df = DataFrame(data=subclonal_structure, columns=['cluster', 'n_ssms', 'proportion', 'ccf'])
        ss_df['ccf'] = pd.to_numeric(ss_df['ccf']) # for allowing to calculate proportion
        ss_df['proportion'] = ss_df['ccf'] * purity

        ma_df = DataFrame(data=mutation_assignments, columns=['id', 'chr', 'pos', 'cluster', 'ccf'], 
                          index=mutation_assignments[:,0])

        ma_df['id'] = pd.to_numeric(ma_df['id']) # for sorting (to retain the original mutation ordering)
        ma_df.sort_values(by=['id'], inplace=True)
        ma_df.drop(columns=['id'], inplace=True)

        ma_df['ccf'] = pd.to_numeric(ma_df['ccf']) # for allowing to calculate proportion
        ma_df['proportion'] = ma_df['ccf'] * purity

        ss_df.to_csv(path_or_buf="{}/{}_subclonal_structure.txt".format(PATH_SUBCLONAL, sample), sep='\t', index=False)
        ma_df.to_csv(path_or_buf="{}/{}_mutation_assignments.txt".format(PATH_MUTASS, sample), sep='\t', index=False)
        
        running_time = time.time() - start_time
        logger.info("{} done in {} (s)".format(sample, running_time))
        print("{} done in {} (s)".format(sample, running_time))
        
    except IOError as e:
        logger.warning(str(e))
        logger.warning("{} possibly consists of too many polyclonal trees, not enough to report a good posterior".format(sample))
        failed_samples.append(sample)
        print(str(e))
        print("{} possibly consists of too many polyclonal trees, not enough to report a good posterior".format(sample))
        
logger.info("Failed samples:\n{}".format(failed_samples))
print("Failed samples:\n{}".format(failed_samples))

Sim_500_194 done in 0.0394220352173 (s)
Sim_500_193 done in 0.0799269676208 (s)
Sim_500_170 done in 0.0903780460358 (s)
Sim_500_160 done in 0.0757830142975 (s)
Sim_500_145 done in 0.125951051712 (s)
Sim_500_15 done in 0.0983390808105 (s)
Sim_500_208 done in 0.0415670871735 (s)
Sim_500_123 done in 0.082258939743 (s)
Sim_500_140 done in 0.0658800601959 (s)
Sim_500_207 done in 0.0829701423645 (s)
Sim_500_105 done in 0.0995960235596 (s)
Sim_500_187 done in 0.0578408241272 (s)
Sim_500_182 done in 0.0671780109406 (s)
Sim_500_188 done in 0.0694530010223 (s)
Sim_500_13 done in 0.104154109955 (s)
Sim_500_197 done in 0.101958036423 (s)
Sim_500_174 done in 0.101083040237 (s)
Sim_500_153 done in 0.062420129776 (s)
Sim_500_162 done in 0.0905539989471 (s)
Sim_500_125 done in 0.0489280223846 (s)
Sim_500_192 done in 0.0778419971466 (s)
Sim_500_157 done in 0.045933008194 (s)
Sim_500_168 done in 0.0881640911102 (s)
Sim_500_110 done in 0.139691114426 (s)
Sim_500_178 done in 0.0454721450806 (s)
Sim_500_10